
# Calibrate Spike Rates

This file can be used to investigate cell spiking behaviour in response to stimuli, in order to get the right firing rate (`pmr/mmr`) in the network files for [parasol](03_network/ms_network_no_m.py) and [midget](03_network/ms_network_no_p.py) cells. It also contains the motion detection stage, so this can be checked as well. 

For testing you probably need to adapt it to simulations, so make sure to set right file paths etc.

In [ ]:
import copy
import numpy as np
import matplotlib.pyplot as plt

### Parameter

Here all parameters are set except $n_D$. Motion detectors are implemented by binning spikes and then comparing different bins in neighboring cells. 

- `t_full`: simulation time
- `t_bin`: interval length for binning in steps (1ms)
- `b_vels`: range of bins that get checked (including the upper limit). These are equivalent to velocity ranges for each bin, cf. [additional information](../../additional_info.ipynb)
- `n_gmdn`: the number of global motion detection neurons into which the simulated cells get divided in $x$- and $y$-direction, division into simple squares
- `npms`: number of neighboring cells that get checked for motion spikes, with the respective delay. For example 2 means that three cells in a row have to spikewith the respective delay fitted to motion velocity and direction.
- `gmd_thrs`: global motion detection threshold $N_g$ 
- `mo_det_thrs`: the minimum number of coincidence events for two intervals, 2 mean that in both cells at the given intervals two spikes must have been detected to spark a motion signal
- `md_num`: if `True`, the numbe rof motion detectors that fired during an interval will be contained in the result, otherwise the result for an intervla will be equal to 1, if any detector detected motion (simple presence of motion)
- `snums`: numbers of the included simulations
- `methods`: `"md_par"` means that the GMDNs are activated by parasol cells, `"md_md"` means by the respective motion detectors. Both can be simulated simultaneously.

In [ ]:
# simulation time
t_full = 1000

#time interval length for binning
t_bin = 15
#range of bins that get checked
b_vels = [3, 5]

#number of 
npms = 1

#this is the sqrt of the number of gmdns used (so how many in one direction)
n_gmdn = np.asarray([1])

#number of cells within RF firing accounted to global motion, set in cells
gmd_thrs = np.arange(100, 99, -1)

#number of coincidence events per bin
mo_det_thrs = np.arange(0, 1, 1)

# number of simulations
snums = [1]

md_num = False

methods = ["md_par", "md_md"]

c = ['blue','orange','green', 'k', 'orangered']

#will contain results, clear here as well
results = {}

Next, we need to set up the grid.

In [ ]:
#get position of parasol neurons for both file sizes
gp_file = open('../data/poletti2010/blocks/1/dot/p_pos_dot_1.data', 'rb')
gp_data = np.load(gp_file)
gp_file.close()

#get first nodes for both to attribute parasol cells right
first_node = 0
gid = open('../data/poletti2010/blocks/1/dot/network/GID_info.txt', 'r+')
for l in gid:
    if 'parasols' in l.split('\t')[0].strip():
        first_node = int(l.split('\t')[1].strip())
f_n = first_node
n_pos_n = {}
swidth = 60.
sheight = 30.
for q in gp_data:
    for p in q:
        n_pos_n[f_n] = (int(p[0]/(swidth/float(n_gmdn))), 0) #(0, 0) 
        f_n += 1

In [ ]:
#names of included blocks, border (refernce frame) has to come first for comparison to work.
snames = ["border", "dot"]

### Simulation of Motion Detection Mechanism

This block gets the input form the file and runs the whole motion detection stage for each simulation. It then plots the IDs of cells, spikes and the motion signal.

In [ ]:
for thr in gmd_thrs:

    gmd_thr = thr

    for md_thr in mo_det_thrs:

        print('GDMN Nr ' + str(n_gmdn) + ' (' + str(int(len(gp_data)*len(gp_data[0])/(n_gmdn))) + 
              ' cells) GMDN threshold ' + str(thr) + ' cells, motion threshold ' + str(md_thr) + ' spikes.')

        in_data = {}
        bin_data = {}
        
        #iterate through all simulations
        for snum in snums:
            
            snum_name = str(snum)
            if snum == 10:
                snum_name = "fem_comp"

            print("simulation number " + str(snum))
            
            #open all four files and run, then store results in dict
            for sname in snames:
                in_data[(snum, sname)] = open('../data/poletti2010/blocks/' + snum_name + '/' + sname +
                                              '/network/spikes_parasols_' + sname + '_' +
                                              snum_name + '.txt', 'r+')

                if sname.count("_m") > 0:
                    n_pos = n_pos_m
                else:
                    n_pos = n_pos_n
                    
                
                fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14., 8.))

                plt.suptitle(r'Spikes NR' + str(snum))
                lables = np.arange(t_bin, t_full+t_bin, t_bin)

                pos = []
                sp = []

                sp_pos = {}
                p = 0
                
                #extract the spike times from file
                for line in in_data[(snum, sname)]:
                    if len(line.split('[')) > 1:
                        p = int(line.split('[')[0])
                        spikes = line.split('[')[1]
                    if len(line.split(']')) > 0:
                        spikes = spikes.split(']')[0]
                        
                    #convert spike times to bin numbers 
                    sp_n = []
                    for s in spikes.split(' '):
                        if len(s.strip()) > 0:
                            pos += [p]
                            sp += [float(s)]
                            sp_n += [int((float(s)-0.01)/t_bin)]
                    #add position to 
                    sp_pos[p] = sp_n                        

                in_data[(snum, sname)].close()


                #store the number of parasol cells firing for each interval
                bins_par_num = np.zeros(shape=(1, 1, int(t_full/t_bin) + 1))
                bins_r_num = np.zeros(shape=(1, 1, int(t_full/t_bin) + 1))
                bins_l_num = np.zeros(shape=(1, 1, int(t_full/t_bin) + 1))

                #iterate through parasols
                for key in sp_pos:

                    #to check the number of parasol cells spiking within one bin
                    did_spike = list(set(sp_pos[key]))
                    #get number of parasol cells spiking wihtin one bin
                    for d_s in did_spike:
                        if sp_pos[key].count(d_s) > md_thr:
                            bins_par_num[n_pos[key][0]][n_pos[key][1]][d_s] += 1

                    #get the number of motion detectors firing        
                    for d_s in did_spike:
                        
                        #rightward motion
                        fs = []
                        for npm in range(npms):
                            fs += [0]
                            #if cell fired at least once during simulaton time
                            if key - (npm + 1) in sp_pos:
                                #check specific intervals
                                for b_vel in range((npm  + 1) * b_vels[0], (npm  + 1) * b_vels[1] + 1):
                                    if (sp_pos[key].count(d_s) > md_thr and 
                                        sp_pos[key - (npm + 1)].count(d_s - b_vel) > md_thr):

                                        #set to 1, if 
                                        fs[npm] = 1

                        #leftward motion
                        fs_l = []
                        for npm in range(npms):
                            fs_l += [0]
                            if key + (npm + 1) in sp_pos:
                                for b_vel in range((npm  + 1) * b_vels[0], (npm + 1) * b_vels[1] + 1):
                                    if (sp_pos[key].count(d_s) > md_thr and 
                                        sp_pos[key + (npm + 1)].count(d_s - b_vel) > md_thr):

                                        fs_l[npm] = 1

                        #if motion detectors in both directions simultaneously detect a motion signal 
                        #-> suppress
                        if 0 not in fs and 0 in fs_l:
                            bins_r_num[n_pos[key][0]][n_pos[key][1]][d_s] += 1

                        if 0 not in fs_l and 0 in fs:
                            bins_l_num[n_pos[key][0]][n_pos[key][1]][d_s] += 1



                '''
                #GLOBAL MOTION DETECTION -- SKIPPED
                gmdns_par = (bins_par_num < gmd_thr).astype(int)
                gmdns_r = (bins_r_num < gmd_thr).astype(int)
                gmdns_l = (bins_l_num < gmd_thr).astype(int)     
                '''
                
                #If assuming one GMDN for both objects (>10deg) -> approximation, as few spikes for dot
                if sname.count("border") > 0:
                    if sname.count("_m") == 0:
                        g_b_par = copy.deepcopy((bins_par_num < gmd_thr).astype(int))
                        g_b_r = copy.deepcopy((bins_r_num < gmd_thr).astype(int))
                        g_b_l = copy.deepcopy((bins_l_num < gmd_thr).astype(int))
                        
                    else:
                        g_b_par_m = copy.deepcopy((bins_par_num < gmd_thr).astype(int))
                        g_b_r_m = copy.deepcopy((bins_r_num < gmd_thr).astype(int))
                        g_b_l_m = copy.deepcopy((bins_l_num < gmd_thr).astype(int))

                if sname.count("_m") > 0:       
                    gmdns_par = g_b_par_m
                    gmdns_r = g_b_r_m
                    gmdns_l = g_b_l_m
                    
                else:
                    gmdns_par = g_b_par
                    gmdns_r = g_b_r
                    gmdns_l = g_b_l
                # end for approximation    
                
                
                #if number of cells spiking in output
                if md_num:
                    bins_par_on_gmd = bins_par_num  * gmdns_par
                    bins_r_on_gmd_par = bins_r_num * gmdns_par
                    bins_l_on_gmd_par = bins_l_num * gmdns_par
                    bins_r_on_gmd_r = bins_r_num * gmdns_r
                    bins_l_on_gmd_l = bins_l_num * gmdns_l
                
                else:
                    bins_r_on_gmd_par = (bins_r_num > 0).astype(int) * gmdns_par
                    bins_l_on_gmd_par = (bins_l_num > 0).astype(int) * gmdns_par
                    bins_r_on_gmd_r = (bins_r_num > 0).astype(int) * gmdns_r
                    bins_l_on_gmd_l = (bins_l_num > 0).astype(int) * gmdns_l

                #difference in leftward vs. rightward motion.
                bins_lr_on_gmd_par = np.abs(bins_r_on_gmd_par - bins_l_on_gmd_par) 
                bins_lr_on_gmd_each = np.abs(bins_r_on_gmd_r - bins_l_on_gmd_l)


                #with several GMDNs, add data from each for each interval    
                bins_lr_on_gmd_par = np.add.reduce(bins_lr_on_gmd_par, 0)    
                bins_lr_on_gmd_par = np.add.reduce(bins_lr_on_gmd_par, 0)
                bins_lr_on_gmd_each = np.add.reduce(bins_lr_on_gmd_each, 0)    
                bins_lr_on_gmd_each = np.add.reduce(bins_lr_on_gmd_each, 0)

                
                #plot spike diagram of parasols
                axes[0][0].plot(pos, sp, 'x', color = c[0])
                axes[0][0].grid()

                #plot parasol cell bins
                #axes[1][0].plot(lables, bins_par_on_gmd, color = c[0])

                axes[1][0].plot(lables, bins_r_on_gmd_par[0][0], color = c[1], label="rightward")
                axes[1][0].plot(lables, bins_l_on_gmd_par[0][0], color = c[2], label="leftward")
                axes[1][0].grid()

                #plot motion detector l/r number of spikes bins
                #axes[0][1].plot(lables, bins_par_num, color = c[0])

                axes[0][1].plot(lables, bins_r_num[0][0], color = c[1], label="rightward")
                axes[0][1].plot(lables, bins_l_num[0][0], color = c[2], label="leftward")

                axes[0][1].plot(lables, bins_par_num[0][0], linestyle=":", color = c[1])
                #axes[0][1].plot(lables, bins_lr_num, linestyle=":", color = c[3], label="diff")
                axes[0][1].grid()
                axes[0][1].legend()

                #plot motion detector l/r motion bins 
                axes[1][1].plot(lables, bins_lr_on_gmd_par, linestyle=":", color = c[2])
                axes[1][1].plot(lables, bins_lr_on_gmd_each, linestyle=":", color = c[3])
                axes[1][1].plot(lables, bins_lr_on_gmd_par, linestyle=":", color = c[4])
                axes[1][1].grid()



                #plt.save()

                plt.show()